Import Statements

In [22]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

Load in the Env and our API Key

In [23]:
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


Beautiful Soup Web Parser Class

In [34]:
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given URL using the BeautifulSoup library.
        """
        self.url = url
        response = requests.get(url, headers=headers)

        # Parse the response content
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # Check if the body tag exists
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = "No body content found."


System Prompt

In [36]:
# The system prompt is what explains the context of the situation to the frontier model
# It tells them what kind of task they are performing and what tone to use

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a cover letter, ignoring text that might be navigation related. \
Respond in markdown."

User Prompt

In [38]:
# A function that writes a User Prompt that asks for summaries of websites:
# The user prompt is the actual conversation itself
# The converstaion start and the role of the LLM is to figure out what way to respond to the user prompt in the context of the system prompt

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a cover letter in markdown. \
Please list off his skills and summarize.\n\n"
    user_prompt += website.text
    return user_prompt

Message Function

In [39]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

Summmary Function

In [40]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

Display Summmary Function

In [41]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [42]:
display_summary("https://www.linkedin.com/in/sagnikdakshit/")

# Cover Letter

[Your Name]  
[Your Address]  
[City, State, Zip]  
[Your Email]  
[Your Phone Number]  
[Date]  

[Employer's Name]  
[Company's Name]  
[Company's Address]  
[City, State, Zip]  

Dear [Employer's Name],

I am writing to express my interest in the [specific position] at [Company's Name]. Although the website did not provide specific content or details about the role or company, I believe my diverse skill set and experiences make me a strong candidate for consideration.

### Skills Summary

While I cannot list specific skills from the website due to the absence of content, here is a general idea of the skills and strengths I bring:

- **Adaptability**: Ability to adjust to new challenges quickly and effectively.
- **Collaboration**: Experience working well in teams, fostering cooperation and understanding.
- **Problem-Solving**: Strong analytical skills to identify issues and create effective solutions.
- **Communication**: Excellent verbal and written communication skills, tailored to various audiences.
- **Time Management**: Proficient in prioritizing tasks and managing time efficiently to meet deadlines.

I would love the opportunity to discuss how my abilities could contribute to your team. Thank you for considering my application. I look forward to the possibility of working together to achieve [Company's Name]'s goals.

Sincerely,  
[Your Name]  